In [ ]:
import numpy as np
import pandas as pd
import statsmodels as sm
import matplotlib.pylab as plt
import config as cf

from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARMA, ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from chinese_calendar import is_workday, is_holiday
from jupyterthemes import jtplot

jtplot.style()
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 200

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))

In [ ]:
train_fix_df = pd.read_csv(cf.train_fix_file_path)
fs_train_df = pd.read_csv(cf.fusai_train_file_path, sep='\t')
# A
fs_testA_df = pd.read_csv(cf.fusai_testA_file_path, sep='\t')
# B
fs_testB_df = pd.read_csv(cf.fusai_testB_file_path, sep='\t')
fs_test_df = fs_testA_df#.append(fs_testB_df)

In [ ]:
f = plt.figure(figsize=(24,3))
ax1 = f.add_subplot(111)
fs_train_brand_df = fs_train_df.groupby('brand', as_index=False)
for frame in fs_train_brand_df:
    frame[1].loc[:,"cnt"].plot(ax = ax1)

In [ ]:
def fix_data_date(df, last_dow, start_date):
    fix_rows = []
    fix_date = 0
    fix_columns = np.append(df.columns.values, 'date_fix')
    is_train = 'cnt' in fix_columns
    for i, row in df.iterrows():
        curr_dow = row['day_of_week']
        while curr_dow - last_dow > 1 or -6 < curr_dow - last_dow <= 0:
            last_dow = last_dow + 1 if last_dow < 7 else 1
            fix_date += 1
            if is_train:
                fix_rows.append(np.array([-1, 0, last_dow, start_date + timedelta(days=fix_date)]))
            else:
                fix_rows.append(np.array([-1, last_dow, start_date + timedelta(days=fix_date)]))
        last_dow = curr_dow
        fix_date += 1
        fix_rows.append(np.append(row.values, start_date + timedelta(days=fix_date)))
    new_df = pd.DataFrame(fix_rows, columns=fix_columns)
    return new_df

# process train date
start_train_date = datetime(2012,12,31)
fs_train_cnt_df = fs_train_df.groupby('date', as_index=False)['cnt'].sum()
fs_train_cnt_df = fs_train_cnt_df.merge(fs_train_df[['date', 'day_of_week']].drop_duplicates(), on='date')
fs_train_new_df = fix_data_date(fs_train_cnt_df, 1, start_train_date)
# process test date
start_test_date = fs_train_new_df.iloc[-1,-1] - timedelta(days=1)
last_dow = fs_train_new_df.iloc[-1,-2] - 1
fs_test_cnt_df = fs_test_df[['date', 'day_of_week']].drop_duplicates()
fs_test_new_df = fix_data_date(fs_test_cnt_df, last_dow, start_test_date)
# A
##test_new_df = test_new_df.append(pd.DataFrame([[-1,3,datetime(2017,3,1)]], columns=test_new_df.columns.values))
# B
#test_new_df = test_new_df.append(pd.DataFrame([[-1,3,datetime(2017,11,29)]], columns=test_new_df.columns.values))
#test_new_df = test_new_df.append(pd.DataFrame([[-1,4,datetime(2017,11,30)]], columns=test_new_df.columns.values))

In [ ]:
fs_train_new_df.to_csv(cf.fusai_train_fix_file_path)

f = plt.figure(figsize=(24,3))
ax1 = f.add_subplot(111)
fs_train_new_df[['cnt']].plot(ax=ax1)
train_fix_df[['cnt']].plot(ax=ax1)

In [ ]:
start_date = '2016-04-05'
fs = fs_train_new_df.loc[fs_train_new_df['date_fix'] > start_date].sum()['cnt']
cs = train_fix_df.loc[(train_fix_df['date_fix'] > start_date) & (train_fix_df['date_fix'] < '2016-05-06')].sum()['cnt']
factor = float(fs) / cs
factor

In [ ]:
start_date_num = fs_train_new_df.loc[fs_train_new_df['date_fix'] == start_date]
start_date_num = start_date_num.iloc[0][0]
print start_date_num

brand_factors = fs_train_df.loc[fs_train_df['date'] > start_date_num].groupby('brand', as_index=False).sum()[['brand', 'cnt']]
brand_factors.loc[:,'factor'] = brand_factors.loc[:,'cnt'] / fs
brand_factors

In [ ]:
# cnt
fs_test_new_df.loc[:,'date_fix'] = fs_test_new_df.loc[:,'date_fix'].astype(str)
train_fix_df.loc[:,'date_fix'] = train_fix_df.loc[:,'date_fix'].astype(str)
fs_test_cnt_df = fs_test_new_df.merge(train_fix_df[['date_fix', 'cnt']], how='left', on='date_fix')
fs_test_cnt_df.loc[:, 'cnt'] = fs_test_cnt_df.loc[:, 'cnt'] * factor
# brand cnt
fs_test_ret_df = fs_test_df.merge(fs_test_cnt_df[['date', 'cnt']], how='left', on='date')
fs_test_ret_df = fs_test_ret_df.merge(brand_factors[['brand', 'factor']], how='left', on='brand')
fs_test_ret_df.loc[:, 'predict'] = fs_test_ret_df.loc[:, 'cnt'] * fs_test_ret_df.loc[:, 'factor'] 

In [ ]:
fs_test_ret_df.to_csv('fs_test_fix.csv')
fs_test_ret_df[['date', 'brand', 'predict']].astype(int).to_csv('fs_test.csv', sep='\t', index=False, header=False)